In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler

import tensorflow as tf
print(f" Found and Using {len(tf.config.experimental.list_physical_devices('GPU'))} GPU")

 Found and Using 1 GPU


In [2]:
def scale_data(df):

    scaled_features = StandardScaler().fit_transform(df.values)
    df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
 
    return(df)

In [3]:
df = pd.read_csv('data/tab_df.csv')
print(len(df))
#df = df.drop(['month', 'day', 'hour', 'min'], axis=1)
df.head(10)

74407


,tick,diff,gain,loss,avg_gain,avg_loss,rs,rsi,ssma,lsma,sema,lema,target
0,1.14907,-0.00012,0.00000,0.00012,0.000326,0.000021,15.724138,94.020619,1.147613,1.145565,1.147735,1.145594,same
1,1.14917,0.00010,0.00010,0.00000,0.000333,0.000016,20.260870,95.296524,1.147931,1.145597,1.147996,1.145676,increase
2,1.14960,0.00043,0.00043,0.00000,0.000352,0.000016,21.434783,95.542636,1.148222,1.145633,1.148287,1.145765,decrease
3,1.14912,-0.00048,0.00000,0.00048,0.000324,0.000051,6.394366,86.476190,1.148446,1.145663,1.148439,1.145841,same
4,1.14917,0.00005,0.00005,0.00000,0.000294,0.000051,5.788732,85.269710,1.148670,1.145694,1.148572,1.145916,decrease
5,1.14885,-0.00032,0.00000,0.00032,0.000278,0.000074,3.776699,79.065041,1.148853,1.145722,1.148622,1.145983,same
6,1.14884,-0.00001,0.00000,0.00001,0.000228,0.000074,3.067308,75.413712,1.148988,1.145750,1.148662,1.146047,decrease
7,1.14870,-0.00014,0.00000,0.00014,0.000214,0.000084,2.542373,71.770335,1.149009,1.145776,1.148669,1.146106,decrease
8,1.14829,-0.00041,0.00000,0.00041,0.000211,0.000114,1.855346,64.977974,1.149000,1.145798,1.148600,1.146155,same
9,1.14829,0.00000,0.00000,0.00000,0.000204,0.000114,1.798742,64.269663,1.148910,1.145819,1.148544,1.146203,increase


In [4]:
mapping = {'decrease': 0, 'same': 1, 'increase': 2}
df = df.replace({'target': mapping})

In [5]:
target_col = 'target'

train_x, valid_x, train_y, valid_y = train_test_split(df.loc[:, df.columns != target_col], df[target_col],train_size = 0.8,random_state = 1)

In [7]:
#train_x = scale_data(train_x)
#valid_x = scale_data(valid_x)
#train_x.head()

In [6]:
Layer_nodes = 512

In [7]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(Layer_nodes, activation='relu', input_shape=[len(train_x.keys())]),
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),              
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',    
    metrics=['accuracy'])

In [9]:
epoch_val = 3

In [10]:
model.fit(train_x, train_y, epochs=epoch_val)

Epoch 1/3
1861/1861 [==============================] - 18s 10ms/step - loss: 0.2823 - accuracy: 0.5449
Epoch 2/3
1861/1861 [==============================] - 17s 9ms/step - loss: -0.0102 - accuracy: 0.5449
Epoch 3/3
1861/1861 [==============================] - 17s 9ms/step - loss: 0.0214 - accuracy: 0.5449


In [13]:
test_loss, test_accuracy = model.evaluate(valid_x, valid_y)
print(f'Test Loss {test_loss}')
print(f'Test Accuracy {test_accuracy}')

156/156 [==============================] - 0s 2ms/step - loss: -472.5865 - accuracy: 0.3605
Test Loss -472.5864562988281
Test Accuracy 0.36052367091178894


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier()
clf.fit(train_x, train_y)

target_names = ['decrease', 'same', 'increase']
print(classification_report(valid_y, clf.predict(valid_x),target_names=target_names))